# NBA Season

### Data Initialization

We are pulling the NBA season stats throughout the years from the URL of https://www.kaggle.com/datasets/justinas/nba-players-data/data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from itertools import combinations
%matplotlib inline

**Get the kaggle dataset**

In [2]:
# read a csv file into a df
playerData = pd.read_csv('nba.csv')

teamNames = pd.read_csv('unique_teams.csv')

playerData.head()

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,0,Randy Livingston,HOU,22.0,193.04,94.800728,Louisiana State,USA,1996,2,...,3.9,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996-97
1,1,Gaylon Nickerson,WAS,28.0,190.50,86.182480,Northwestern Oklahoma,USA,1994,2,...,3.8,1.3,0.3,8.9,0.030,0.111,0.174,0.497,0.043,1996-97
2,2,George Lynch,VAN,26.0,203.20,103.418976,North Carolina,USA,1993,1,...,8.3,6.4,1.9,-8.2,0.106,0.185,0.175,0.512,0.125,1996-97
3,3,George McCloud,LAL,30.0,203.20,102.058200,Florida State,USA,1989,1,...,10.2,2.8,1.7,-2.7,0.027,0.111,0.206,0.527,0.125,1996-97
4,4,George Zidek,DEN,23.0,213.36,119.748288,UCLA,USA,1995,1,...,2.8,1.7,0.3,-14.1,0.102,0.169,0.195,0.500,0.064,1996-97


**Put abbreviations to Cities**

Some cities have multiple as franchises have changed

Cities had to be added as ESPN win rates used cities

In [3]:
# merge the two dataframes on team_abbreviation with df and dfTeams on abbreviations
playerData = pd.merge(playerData, teamNames, left_on='team_abbreviation', right_on='abbreviations')

# drop the team_abbreviation column and abbreviations column
playerData = playerData.drop(columns=['team_abbreviation', 'abbreviations'])

In [4]:
playerData.head()

,Unnamed: 0,player_name,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,team
0,0,Randy Livingston,22.0,193.04,94.800728,Louisiana State,USA,1996,2,42,...,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996-97,Houston
1,18,Hakeem Olajuwon,34.0,213.36,115.665960,Houston,Nigeria,1984,1,1,...,9.2,3.0,6.5,0.075,0.206,0.308,0.558,0.158,1996-97,Houston
2,29,Emanual Davis,28.0,195.58,87.996848,Delaware State,USA,Undrafted,Undrafted,Undrafted,...,1.7,2.0,6.6,0.011,0.098,0.144,0.565,0.191,1996-97,Houston
3,61,Joe Stephens,24.0,200.66,95.254320,Arkansas-Little Rock,USA,Undrafted,Undrafted,Undrafted,...,1.5,0.0,-17.4,0.250,0.111,0.279,0.300,0.000,1996-97,Houston
4,97,Eddie Johnson,38.0,200.66,97.522280,Illinois,USA,1981,2,29,...,2.7,1.0,4.1,0.034,0.126,0.220,0.541,0.102,1996-97,Houston


In [5]:
# drop the unnamed column
playerData.drop('Unnamed: 0', axis=1, inplace=True)

playerData.dtypes

player_name       object
age              float64
player_height    float64
player_weight    float64
college           object
country           object
draft_year        object
draft_round       object
draft_number      object
gp                 int64
pts              float64
reb              float64
ast              float64
net_rating       float64
oreb_pct         float64
dreb_pct         float64
usg_pct          float64
ts_pct           float64
ast_pct          float64
season            object
team              object
dtype: object

In [6]:
# look for null values
playerData.isnull().sum()

player_name         0
age                 0
player_height       0
player_weight       0
college          1852
country             0
draft_year          0
draft_round         0
draft_number        0
gp                  0
pts                 0
reb                 0
ast                 0
net_rating          0
oreb_pct            0
dreb_pct            0
usg_pct             0
ts_pct              0
ast_pct             0
season              0
team                0
dtype: int64

### Data Preprocessing

We need to deal with the columns we want to keep and also all the categorial data cols of:

player_name           object

team_abbreviation     object

college               object

country               object

draft_year            object

draft_round           object

draft_number          object

season                object

#### **Drop Some of Them**

In [7]:
playerData.drop(['player_name', 'college', 'draft_year', 'draft_round', 'draft_number', 'country'], axis=1, inplace=True)

In [8]:
playerData.head()

,age,player_height,player_weight,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,team
0,22.0,193.04,94.800728,64,3.9,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996-97,Houston
1,34.0,213.36,115.665960,78,23.2,9.2,3.0,6.5,0.075,0.206,0.308,0.558,0.158,1996-97,Houston
2,28.0,195.58,87.996848,13,5.0,1.7,2.0,6.6,0.011,0.098,0.144,0.565,0.191,1996-97,Houston
3,24.0,200.66,95.254320,2,1.5,1.5,0.0,-17.4,0.250,0.111,0.279,0.300,0.000,1996-97,Houston
4,38.0,200.66,97.522280,52,8.2,2.7,1.0,4.1,0.034,0.126,0.220,0.541,0.102,1996-97,Houston


In [9]:
# categorical code season col but i want to keep the original
playerData['season'] = pd.Categorical(playerData['season']).codes + 1997

# drop the rows where seasonEncoded is less than 7. This keeps the season of 03-04 and later
playerData = playerData[playerData['season'] >= 2004]

playerData.reset_index(drop=True, inplace=True)

#### **Add in Win Rates**

In [10]:
def winRateFromYear(year):

    winRateDf = pd.read_html(f'https://www.teamrankings.com/nba/stat/win-pct-all-games?date={year}-06-16')[0]

    winRateDf['Win PCT']= winRateDf[f'{year - 1}'] 

    winRateDf['season'] = year

    winRateDf = winRateDf[['Team', 'Win PCT', 'season']]

    return winRateDf

**Merge DF2 with df on Team Names**

In [11]:
def getWinRates():
    # merge the two dataframes on team with df and teams on team where season is 2004
    winRateDf = pd.DataFrame()

    for year in range(2004, 2024):
        winRateDf = pd.concat([winRateDf, winRateFromYear(year)], ignore_index=True)

    winRateDf.tail()

    return winRateDf

winRateDf = pd.read_csv('winRate.csv')


In [12]:
winRateDf.head()

,Team,Win PCT,season
0,Indiana,0.725,2004
1,San Antonio,0.685,2004
2,Minnesota,0.680,2004
3,Detroit,0.667,2004
4,LA Lakers,0.664,2004


**TODO: MERGE THE DFS**

In [13]:
updatedPlayerData = pd.merge(playerData, winRateDf, left_on=['team', 'season'], right_on=['Team', 'season'])

updatedPlayerData.drop(['Team'], axis=1, inplace=True)

updatedPlayerData.head()

,age,player_height,player_weight,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,team,Win PCT
0,40.0,205.74,111.130040,7,1.3,0.7,0.3,-7.9,0.000,0.217,0.165,0.521,0.100,2004,Houston,0.529
1,33.0,200.66,122.469840,52,5.0,3.9,0.6,-5.0,0.104,0.175,0.152,0.538,0.063,2004,Houston,0.529
2,28.0,193.04,97.522280,80,15.8,4.5,3.2,1.8,0.015,0.112,0.200,0.535,0.144,2004,Houston,0.529
3,23.0,205.74,100.243832,45,3.1,1.6,0.7,2.0,0.021,0.136,0.159,0.477,0.103,2004,Houston,0.529
4,29.0,195.58,104.326160,19,0.6,1.0,0.5,-8.4,0.010,0.167,0.093,0.278,0.132,2004,Houston,0.529


In [14]:
updatedPlayerData['AVG Minutes Played'] = updatedPlayerData['gp'] * updatedPlayerData['usg_pct']

In [15]:
# Filter the df to only include the max 5 of games played from each seasonEncoded and Team
updatedPlayerData = updatedPlayerData.groupby(['team', 'season']).apply(lambda x: x.nlargest(5, 'AVG Minutes Played')).reset_index(drop=True)

updatedPlayerData.head()

,age,player_height,player_weight,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,team,Win PCT,AVG Minutes Played
0,26.0,203.20,98.883056,80,18.1,4.6,3.1,-3.8,0.038,0.102,0.249,0.521,0.155,2004,Atlanta,0.342,19.920
1,26.0,187.96,81.646560,81,16.8,4.1,5.4,-4.0,0.019,0.106,0.231,0.519,0.261,2004,Atlanta,0.342,18.711
2,31.0,195.58,90.718400,80,7.5,4.1,2.9,0.8,0.071,0.158,0.191,0.510,0.240,2004,Atlanta,0.342,15.280
3,29.0,205.74,106.594120,56,10.2,3.1,0.8,-5.2,0.054,0.117,0.214,0.544,0.068,2004,Atlanta,0.342,11.984
4,29.0,185.42,86.182480,71,3.8,1.6,2.7,-9.8,0.011,0.094,0.143,0.440,0.256,2004,Atlanta,0.342,10.153


#### **Convert Player Stats into Starters' Team Stats**

In [16]:
teamData = updatedPlayerData.groupby(['season', 'team']).agg(
    ptsTotal=('pts', 'sum'),
    rebTotal=('reb', 'sum'),
    astTotal=('ast', 'sum'),
    MinutesPlayed=('AVG Minutes Played', 'sum'),
    averageAge=('age', 'mean'),
    averageHeight=('player_height', 'mean'),
    averageWeight=('player_weight', 'mean'),
    winRate=('Win PCT', 'first')
).reset_index()

teamData

,season,team,ptsTotal,rebTotal,astTotal,MinutesPlayed,averageAge,averageHeight,averageWeight,winRate
0,2004,Atlanta,56.4,17.5,14.9,76.048,28.2,195.580,92.804923,0.342
1,2004,Boston,58.8,21.7,13.2,87.015,24.0,200.152,99.608803,0.419
2,2004,Brooklyn,71.6,30.0,20.0,80.431,28.8,199.136,100.062395,0.581
3,2004,Chicago,62.5,24.9,16.1,77.090,27.8,199.644,100.788142,0.281
4,2004,Cleveland,67.7,31.5,11.6,82.232,26.0,207.772,110.313574,0.427
...,...,...,...,...,...,...,...,...,...,...
554,2023,Sacramento,87.8,26.9,21.8,85.421,26.4,200.152,95.072883,0.573
555,2023,San Antonio,64.1,19.4,17.0,67.671,23.8,194.056,93.439952,0.268
556,2023,Toronto,85.6,26.6,19.8,77.189,26.6,198.120,96.252222,0.494
557,2023,Utah,83.9,24.2,16.7,73.005,27.0,200.152,99.608803,0.451


In [17]:
winRate = teamData['winRate']

In [18]:
teamData.drop(['season','team','winRate'], axis=1, inplace=True)

teamData.head()

,ptsTotal,rebTotal,astTotal,MinutesPlayed,averageAge,averageHeight,averageWeight
0,56.4,17.5,14.9,76.048,28.2,195.580,92.804923
1,58.8,21.7,13.2,87.015,24.0,200.152,99.608803
2,71.6,30.0,20.0,80.431,28.8,199.136,100.062395
3,62.5,24.9,16.1,77.090,27.8,199.644,100.788142
4,67.7,31.5,11.6,82.232,26.0,207.772,110.313574


In [19]:
winRate.head()

0    0.342
1    0.419
2    0.581
3    0.281
4    0.427
Name: winRate, dtype: float64

In [20]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(teamData, winRate, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test_Scaled = scaler.transform(X_test)

In [21]:
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),  # Input layer
    keras.layers.Dense(64, activation='relu'),       # Hidden layer
    keras.layers.Dense(32, activation='relu'),       # Hidden layer
    keras.layers.Dense(1)                             # Output layer (for regression)
])

In [22]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [23]:
model.fit(X_train, y_train, epochs=100, batch_size=5, validation_split=0.2, verbose=0)

In [24]:
# Example input for a new set of NBA players

predicted_win_rates = model.predict(X_test_Scaled)
actual_win_rates = y_test.values

# Display predictions alongside actual values
for pred, actual in zip(predicted_win_rates.flatten(), actual_win_rates):
    print(f'Predicted: {pred:.3f}, Actual: {actual:.3f}')

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Predicted: 0.693, Actual: 0.679
Predicted: 0.267, Actual: 0.339
Predicted: 0.304, Actual: 0.317
Predicted: 0.382, Actual: 0.750
Predicted: 0.392, Actual: 0.471
Predicted: 0.678, Actual: 0.511
Predicted: 0.639, Actual: 0.354
Predicted: 0.546, Actual: 0.500
Predicted: 0.586, Actual: 0.640
Predicted: 0.605, Actual: 0.792
Predicted: 0.472, Actual: 0.293
Predicted: 0.485, Actual: 0.628
Predicted: 0.494, Actual: 0.317
Predicted: 0.688, Actual: 0.725
Predicted: 0.782, Actual: 0.638
Predicted: 0.606, Actual: 0.632
Predicted: 0.774, Actual: 0.644
Predicted: 0.655, Actual: 0.725
Predicted: 0.649, Actual: 0.523
Predicted: 0.556, Actual: 0.596
Predicted: 0.609, Actual: 0.470
Predicted: 0.549, Actual: 0.563
Predicted: 0.381, Actual: 0.506
Predicted: 0.390, Actual: 0.512
Predicted: 0.286, Actual: 0.534
Predicted: 0.529, Actual: 0.613
Predicted: 0.706, Actual: 0.656
Predicted: 0.480, Actual: 0.232
Predicted: 0.408, Actual: 0.402
Predicted: 0.723, Actual: 0.582
Pr

In [25]:
# Calculate regression metrics
mae = mean_absolute_error(actual_win_rates, predicted_win_rates)
mse = mean_squared_error(actual_win_rates, predicted_win_rates)
r2 = r2_score(actual_win_rates, predicted_win_rates)

print(f'Mean Absolute Error: {mae:.2f}')
print(f'Mean Squared Error: {mse:.2f}')
print(f'R-squared: {r2:.2f}')

Mean Absolute Error: 0.11
Mean Squared Error: 0.02
R-squared: 0.14


In [26]:
# Define weights for each statistic
weights = {
    'pts': 0.3,  # Weight for points
    'ast': 0.3,  # Weight for assists
    'reb': 0.4   # Weight for rebounds
}

# Calculate combined score
updatedPlayerData['combined_score'] = (updatedPlayerData['pts'] * weights['pts'] +
                        updatedPlayerData['ast'] * weights['ast'] +
                        updatedPlayerData['reb'] * weights['reb'])

In [33]:
top_players = updatedPlayerData.sort_values(by='combined_score', ascending=False).head(10)

In [34]:
top_players

,age,player_height,player_weight,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,team,Win PCT,AVG Minutes Played,combined_score
1070,29.0,195.58,99.790240,78,36.1,6.6,7.5,6.3,0.023,0.157,0.396,0.616,0.394,2019,Houston,0.634,30.888,21.62
1075,30.0,195.58,99.790240,68,34.3,6.6,7.5,5.8,0.026,0.139,0.356,0.626,0.366,2020,Houston,0.625,24.208,20.72
690,24.0,200.66,104.326160,66,32.4,8.6,8.0,2.1,0.024,0.224,0.368,0.609,0.408,2023,Dallas,0.463,24.288,20.32
1205,27.0,198.12,99.790240,80,35.4,5.3,4.5,4.7,0.026,0.127,0.384,0.559,0.228,2006,LA Lakers,0.539,30.720,20.11
2090,29.0,213.36,127.005760,66,33.1,10.2,4.2,8.8,0.057,0.243,0.370,0.655,0.233,2023,Philadelphia,0.656,24.420,19.85
1590,28.0,213.36,110.222856,63,31.1,11.8,5.7,7.2,0.065,0.268,0.373,0.605,0.314,2023,Milwaukee,0.678,23.499,19.62
1060,27.0,195.58,99.790240,81,29.1,8.1,11.2,6.3,0.035,0.212,0.341,0.613,0.505,2017,Houston,0.656,27.621,19.53
2005,31.0,182.88,74.842680,72,33.0,3.2,7.4,0.8,0.016,0.071,0.354,0.543,0.331,2006,Philadelphia,0.463,25.488,19.36
2290,32.0,187.96,88.450440,58,32.2,4.8,7.3,2.1,0.022,0.112,0.331,0.645,0.338,2023,Portland,0.402,19.198,19.25
1575,25.0,210.82,109.769264,63,29.5,13.6,5.6,15.4,0.068,0.307,0.363,0.613,0.328,2020,Milwaukee,0.815,22.869,19.15


In [35]:
# Get all combinations of 5 players
combinations_of_5 = list(combinations(top_players.index, 5))

In [36]:
print(combinations_of_5)

[(1070, 1075, 690, 1205, 2090), (1070, 1075, 690, 1205, 1590), (1070, 1075, 690, 1205, 1060), (1070, 1075, 690, 1205, 2005), (1070, 1075, 690, 1205, 2290), (1070, 1075, 690, 1205, 1575), (1070, 1075, 690, 2090, 1590), (1070, 1075, 690, 2090, 1060), (1070, 1075, 690, 2090, 2005), (1070, 1075, 690, 2090, 2290), (1070, 1075, 690, 2090, 1575), (1070, 1075, 690, 1590, 1060), (1070, 1075, 690, 1590, 2005), (1070, 1075, 690, 1590, 2290), (1070, 1075, 690, 1590, 1575), (1070, 1075, 690, 1060, 2005), (1070, 1075, 690, 1060, 2290), (1070, 1075, 690, 1060, 1575), (1070, 1075, 690, 2005, 2290), (1070, 1075, 690, 2005, 1575), (1070, 1075, 690, 2290, 1575), (1070, 1075, 1205, 2090, 1590), (1070, 1075, 1205, 2090, 1060), (1070, 1075, 1205, 2090, 2005), (1070, 1075, 1205, 2090, 2290), (1070, 1075, 1205, 2090, 1575), (1070, 1075, 1205, 1590, 1060), (1070, 1075, 1205, 1590, 2005), (1070, 1075, 1205, 1590, 2290), (1070, 1075, 1205, 1590, 1575), (1070, 1075, 1205, 1060, 2005), (1070, 1075, 1205, 1060, 229

In [37]:
createdTeams = []
team = pd.DataFrame()

for combo in combinations_of_5:
    for player in combo:
        # print(player)
        # print(top_players.loc[player].to_frame().T)
        team = pd.concat([team, top_players.loc[player].to_frame().T])

    createdTeams.append(team.agg(
        ptsTotal=('pts', 'sum'),
        rebTotal=('reb', 'sum'),
        astTotal=('ast', 'sum'),
        MinutesPlayed=('AVG Minutes Played', 'sum'),
        averageAge=('age', 'mean'),
        averageHeight=('player_height', 'mean'),
        averageWeight=('player_weight', 'mean'),
    ).reset_index())